In [ ]:
#import libraries
import scipy.io
import numpy as np
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling1D, Flatten, Reshape, Dense,Cropping1D, advanced_activations, BatchNormalization, Concatenate, LSTM, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD
from keras.models import Model, Sequential
from keras import losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import pickle

In [ ]:
#load data in python runtime
# 'data_foot2hip.mat' is the name of a matlab file (varioable) which contains ground truth, outsole and insole data
tempData = scipy.io.loadmat('data_foot2hip.mat')

In [ ]:
#Set fixed initial seed for random values
np.random.seed(0)
#Extract training and testing data from tempData variable
trainX=tempData['data_X_train_norm']
trainY=tempData['data_Y_train_norm']
testX=tempData['data_X_test_norm']
testY=tempData['data_Y_test_norm']
# Reshape the data for further processing
trainX=trainX.reshape(701930,150,7,1) #Make 4D array for CNN layers 
testX=testX.reshape(233970,150,7,1) #Make 4D array for CNN layers 
#each data point of trainX or testX has data of 5 seconds, six distances (foot to ground) and CoP.
#each data point of trainY or testY has data of knee, hip and ankle joint angles.

In [ ]:
#define model the architecture of Foot2hip
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,1), activation='relu',padding='same',input_shape=(150, 7,1))) 
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Conv2D(64, (3, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Conv2D(1, (3, 1), activation='relu', padding='same'))
model.add(Reshape((37,7)))
model.add(LSTM(64, activation='relu', input_shape=(37, trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dense(128,activation='linear'))
model.add(Dense(64,activation='linear'))
model.add(Dense(trainY.shape[1],activation='linear'))

adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=adam, loss='mse')
model.summary()

In [ ]:
# Train the model
checkpointer = ModelCheckpoint(filepath='./foot2hip.hdf5', verbose = 1, save_best_only = True) # verbose =1 is for printing the output
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
history = model.fit(trainX, trainY, epochs=150,batch_size=16, callbacks = [checkpointer,es], validation_split=0.2, verbose=1)

In [ ]:
#training history
#plot
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
#saving the losses
with open('loss', 'wb') as f:
    pickle.dump(history.history['loss'], f)
with open('val_loss', 'wb') as f:
    pickle.dump(history.history['val_loss'], f)

In [ ]:
#De-standardization of the data
X_test_mean=tempData['X_test_mean'] #X_test_mean is mean of test input data
Y_test_mean=tempData['Y_test_mean']#Y_test_mean is mean of test output data
X_test_std=tempData['X_test_std']#X_test_std is mean of test input data
Y_test_std=tempData['Y_test_std']#Y_test_std is mean of test output data
Y_test_std=np.squeeze(Y_test_std)
Y_test_mean=np.squeeze(Y_test_mean)
X_test_mean=X_test_mean.reshape(1,150,7,1)
X_test_std=X_test_std.reshape(1,150,7,1)
X_test_scaled=testX*X_test_std+X_test_mean #De-standardization of the data
Y_test_scaled=testY*Y_test_std+Y_test_mean #De-standardization of the data

In [ ]:
#ploting the predicted and original angle profiles
sample_start=7000 #use any number
prediction = model.predict(testX[sample_start:sample_start+30,0:150,0:7]) 
plt.plot(prediction, color='r')
plt.plot(testY[sample_start:sample_start+30], label='Ground Truth', color='b')
plt.legend()
plt.show()

prediction_scaled=prediction*Y_test_std+Y_test_mean
plt.plot(prediction_scaled, color='r')  
plt.plot(Y_test_scaled[sample_start:sample_start+30], label='Ground Truth', color='b')
plt.legend()